In [158]:
import backtrader as bt
import yfinance as yf
from datetime import datetime

In [159]:
cerebro = bt.Cerebro()

In [160]:
data = bt.feeds.PandasData(dataname=yf.download('AAPL', '2010-01-01','2017-01-01'))

[*********************100%***********************]  1 of 1 completed


In [161]:
class MyStrategy(bt.Strategy):
    params = (('period_fast',30),('period_slow',200))
    def __init__(self):
        #obtains the data provided.
        self.close_data = self.data.close
        #Create indicators.
        self.fast_sma = bt.indicators.MovingAverageSimple(self.close_data,period=self.params.period_fast)
        self.slow_sma = bt.indicators.MovingAverageSimple(self.close_data,period=self.params.period_slow)
        
    def next(self):
        #Check if already in position
        if not self.position:
            #Open a long position at crossover of MA indicators. (buy)
            if self.fast_sma[0] > self.slow_sma[0] and self.fast_sma[-1] < self.slow_sma[-1]:
                self.buy()
        else:
            #Check whether to close long position. (sell)
            if self.fast_sma[0] < self.slow_sma[0] and self.fast_sma[-1] > self.slow_sma[-1]:
                self.close()
                
        
      

In [162]:
cerebro.adddata(data)
cerebro.addstrategy(MyStrategy)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.SharpeRatio,riskfreerate=0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.broker.set_cash(3000)
print('Initial capital: $%.2f' % cerebro.broker.get_value())
cerebro.broker.setcommission(0.01)

Initial capital: $3000.00


In [163]:
# Running strategy and getting results to evaluate
results = cerebro.run()

In [164]:
print('Sharpe Ratio: %.2f' % results[0].analyzers.sharperatio.get_analysis()['sharperatio'])

Sharpe Ratio: 0.72


In [165]:
print('Returns: %.2f%%' % results[0].analyzers.returns.get_analysis()['rnorm100'])

Returns: 0.06%


In [166]:
print('Max Drawdown: %.2f%%' % results[0].analyzers.drawdown.get_analysis()['max']['drawdown'])

Max Drawdown: 0.21%


In [167]:
print('Capital: $%.2f' % cerebro.broker.get_value())

Capital: $3012.69
